In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a = 1
medium = td.Medium(permittivity=11.56)


In [ ]:
#file = r"H:\phd stuff\structures\SHU_2D\chi_0.20_N484_posics.dat"
file = r"H:\phd stuff\structures\SHU_2D\chi_0.45_N484_posics.dat"
data = []
centers = []

with open(file,"r") as f: 
    lines = f.read().splitlines() 
    i=0
    index = 1
    num_samples = int(lines[index].split()[0])

    while num_samples>0 and i < 10:
        try:
            num_samples = int(lines[index].split()[0])
            info = lines[index]
            old_index = index
            index += num_samples+1
            li = (lines[old_index+1:index])
            centers = np.array([list(map(float, item.split())) for item in li])
            data += [{
                "centers":centers,
                "Lx":float(lines[old_index].split()[1]),
                "Ly":float(lines[old_index].split()[2]),
                "points":float(lines[old_index].split()[0]),
            }]
            i+=1
        except:
            num_samples = 0




In [ ]:
lambdas =  a/np.array([0.493,0.495])
project_name = r"PNAS Test 2D Structures T(t) Samples N 480 xi 0.45 Diffusion"
runtime = 6
min_steps_per_lambda = 20

for k,item in enumerate(data):
    #Tight pulse 


    structure_1 = AM.loadAndRunStructure(key = tidy3dAPI
                ,direction="z", lambda_range=lambdas,
                box_size= data[k]['Lx']*a,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda,
               scaling=1,shuoff_condtion=1e-20, verbose=True,
               monitors=["flux"],
               freqs=200, 
               source="planewave", 
               width=0.1, ref_only=True
               )
           
    print(structure_1.__str__())

    sim = structure_1.sim

    

    
    boundaries= td.BoundarySpec(
            x=td.Boundary.periodic(),
            y=td.Boundary(plus=td.Absorber(num_layers=100),minus=td.Absorber(num_layers=100)) ,
            z=td.Boundary.periodic(),
        )
    sim = sim.copy(update={'boundary_spec':boundaries})
    sim = sim.copy(update={'sources':[]})
    sim = sim.copy(update={'monitors':[]})


    sim = sim.copy(update={'size':[data[k]['Lx']*a,30,0]})
    Lx, Ly = data[k]['Lx'],30

    source = td.PlaneWave(
            source_time = td.GaussianPulse(
                freq0=structure_1.freq0,
                fwidth=structure_1.freqw
            ),
            size= (td.inf,0,td.inf),
            center=(0,-14,0),
            direction='+',
            pol_angle=np.pi/2,
            name='planewave',
            )
    
    sim = sim.copy(update={'sources':[source]})

    monitor_1 = td.FluxMonitor(
                center = (
                            0,-12.5,0
                            ),
                size = (
                   td.inf,0,td.inf
                    ),
                freqs = structure_1.monitor_freqs,
                name='flux2' )

    monitor_2 = td.FluxMonitor(
                center = (
                            0,12.5,0
                            ),
                size = (
                   td.inf,0,td.inf
                    ),
                freqs = structure_1.monitor_freqs,
                name='flux1' 
            )

    time_monitorT = td.FluxTimeMonitor(
                    center = (
                            0,12.5,0
                            ),
                size = (
                   td.inf,0,td.inf
                    ),
                    interval = 200,
                    name="time_monitorT",

                )
    
    sim = sim.copy(update={'monitors':[monitor_1,monitor_2,time_monitorT]})
    
    slab_1 = td.Structure(
                    geometry=td.Box(
                        center= (0,(Ly/2+Lx/2)/2,0),
                        size=(td.inf,(Ly/2-Lx/2),td.inf),
                    ),
                    medium=medium,
                    name='slab1',
                    )

    slab_2 = td.Structure(
                    geometry=td.Box(
                        center= (0,-(Ly/2+Lx/2)/2,0),
                        size=(td.inf,(Ly/2-Lx/2),td.inf),
                    ),
                    medium=medium,
                    name='slab2',
                    )
    
    cyl_group = []
    for x,y in data[k]['centers']:
        cyl_group.append(td.Cylinder(center=[x, y, 0], radius=0.189, length=td.inf))

    cylinders = td.Structure(geometry=td.GeometryGroup(geometries=cyl_group), medium=medium)

    sim = sim.copy(update={'structures':[cylinders,slab_1,slab_2]})
    fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
    sim.plot(z=0, ax=ax)
    plt.show()

    

    id =web.upload(sim, folder_name=project_name,task_name=f"pnas_T_t_Localization {1/lambdas[0]:.3g} - {1/lambdas[1]:.3g} - Sample_{k}", verbose=True)
    web.start(task_id = id)
    web.monitor(task_id=id,verbose=True)
    ids = '\n' + id
    incidence_folder = "z_incidence"
    file_path = f"data/{project_name}/{incidence_folder}/pnas_T_t_Localization {1/lambdas[0]:.3g} - {1/lambdas[1]:.3g} - Sample_{k}.txt"
    # Check if the folder exists
    if not os.path.exists( f"data/{project_name}/{incidence_folder}"):
        os.makedirs(f"data/{project_name}/{incidence_folder}")
        print(f"Folder '{project_name}/{incidence_folder}' created successfully.")

    # Open file in write mode
    with open(file_path, "w") as file:
        # Write the string to the file
        file.write(ids)

    sim_data = web.load(id)


    flux_t = sim_data["time_monitorT"].flux
    times = flux_t['t']
    max_index = np.argmax(flux_t.values)
    monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)

    plt.plot(times[max_index:]*1e12,np.log(flux_t[max_index:]/np.max(flux_t[max_index:])),"o",label=f"$\\nu$: {1/monitor_lambdas[0]:.3g} - {1/monitor_lambdas[-1]:.3g}")
    plt.ylabel("Log(T(t))")
    plt.xlabel("Time[ps]")
    plt.title("Normalized transmission T(t)")
    plt.legend()

    plt.grid()
    plt.show()
